In [ ]:
!apt install libpoppler-cpp-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libpoppler-cpp0v5
The following NEW packages will be installed:
  libpoppler-cpp-dev libpoppler-cpp0v5
0 upgraded, 2 newly installed, 0 to remove and 34 not upgraded.
Need to get 36.7 kB of archives.
After this operation, 188 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libpoppler-cpp0v5 amd64 0.62.0-2ubuntu2.12 [28.0 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libpoppler-cpp-dev amd64 0.62.0-2ubuntu2.12 [8,676 B]
Fetched 36.7 kB in 1s (63.2 kB/s)
Selecting previously unselected package libpoppler-cpp0v5:amd64.
(Reading database ... 160690 files and directories currently installed.)
Preparing to unpack .../libpoppler-cp

In [ ]:
!pip install pdftotext
!pip install transformers
!python -m nltk.downloader punkt
!pip install sentencepiece

     |████████████████████████████████| 102kB 6.2MB/s 
  Created wheel for pdftotext: filename=pdftotext-2.1.5-cp37-cp37m-linux_x86_64.whl size=55027 sha256=220778bdc52a1ee39299602bf35c2bb3a8d4ec4a9d80c2ee6f28ba2bd3bf41d7
  Stored in directory: /root/.cache/pip/wheels/59/1d/72/339d726f6737b281a498eb05203256a30f19c54193738996bc
Successfully built pdftotext
     |████████████████████████████████| 2.1MB 8.2MB/s 
     |████████████████████████████████| 3.3MB 49.7MB/s 
     |████████████████████████████████| 901kB 49.2MB/s 
/usr/lib/python3.7/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
     |████████████████████████████████| 1.2MB 8.1MB/s 


In [ ]:
import pdftotext 
import re

class TextProcessingAndContextCreation :

    @classmethod
    def get_context_chunks(cls, file_name) :
        
        raw_text = cls._get_raw_text_from_pdf(file_name)
        first_page_number = cls._get_the_first_page_number(raw_text)
        if first_page_number == -1 :
            print("Error encountered")
            return []
        raw_text = raw_text[first_page_number:]
        cls._remove_header_of_first_page(raw_text)
        cls._delete_footer_notes(raw_text)
        cls._remove_captial_words(raw_text)
        text = cls._join_all_pages(raw_text)
        text = cls._split_the_text_on_bold_points(text)
        cls._filter_and_get_plain_text(text)
        # return text 
        contexts =  cls._get_the_contexts(text)
        return contexts

    @classmethod
    def _get_raw_text_from_pdf(cls, file_name) :

        with open(file_name, "rb") as f :
            pdf_text = pdftotext.PDF(f)
        
        raw_text = []
        for i in pdf_text :
            raw_text.append(str(i))
        return raw_text

    @classmethod
    def _get_the_first_page_number(cls, raw_text) :

        index = raw_text[0].split().index('ACT,')
        match_text_page_zero = " ".join(raw_text[0].split()[:index + 1])

        for i in range(1, len(raw_text)) :
            page_match_text = " ".join(raw_text[i].split()[:index + 1])
            if match_text_page_zero == page_match_text :
                return i
        return -1

    @classmethod
    def _remove_header_of_first_page(cls, raw_text) :

        temp = re.match(r"(.|\n)*?\[.*\]", raw_text[0])
        raw_text[0] = raw_text[0][temp.end():]

    @classmethod
    def _delete_footer_notes(cls, raw_text) :

        footer_regex = r"1\..*"
        for i in range(len(raw_text)) :
            temp = raw_text[i].split('\n')[:-2]
            for j in range(len(temp) - 1, -1, -1) :
                if re.match(footer_regex, temp[j]) is not None :
                    break
            if j != 0 :
                temp = temp[:j]
            raw_text[i] = "\n".join(temp)

    @classmethod
    def _remove_captial_words(cls, raw_text) :

        for i in range(len(raw_text)) :
            raw_text[i] = raw_text[i].split('\n')
            raw_text[i] = list(filter(lambda x: not (x.isupper()), raw_text[i]))
            raw_text[i] = "\n".join(raw_text[i])

    @classmethod
    def _join_all_pages(cls, raw_text) :

        concatenated_text = []
        for i in raw_text :
            concatenated_text.append(i)
        concatenated_text = "\n".join(concatenated_text)
        return concatenated_text

    @classmethod
    def _split_the_text_on_bold_points(cls, text) :

        split_regex = r"[0-9]+[A-Z]*\.[^\n]"
        split_text = re.split(split_regex, text)
        return split_text

    @classmethod
    def _filter_and_get_plain_text(cls, text) :

        for i in range(len(text)) :
            text[i] = re.sub(r"(“|”|’)", "\"", text[i])
            text[i] = re.sub(r"\n", " ", text[i])
            text[i] = re.sub(r" +", " ", text[i])
            text[i] = re.sub(r"—", "- ", text[i])

    @classmethod
    def _get_the_contexts(cls, text) :

        contexts = []
        for i in range(len(text)) :
            if (len(text[i].split()) > 10 and len(text[i].split()) <= 512) :
                contexts.append(text[i])
            else :
                temp = cls._get_contexts_greater_than_max_size(text[i])
                for i in temp :
                    contexts.append(" ".join(i))

        return contexts

    @classmethod
    def _get_contexts_greater_than_max_size(cls, text) :

        ROMAN_NUMERALS = ['i', 'ii', 'iii', 'iv', 'v', 'vi', 'vii', 'viii', 'ix', 'x', \
                          'xi', 'xii', 'xiii', 'xiv', 'xv', 'xvi', 'xvii', 'xviii', 'xix', 'xx', \
                          'xxi', 'xxii', 'xxiii', 'xxiv', 'xxv', 'xxvi', 'xxvii', 'xxviii', 'xxix', 'xxx']
        level_wise_regex = [r"\([0-9]+\)", r"\([a-z]\)", r"\((" + "|".join(ROMAN_NUMERALS) + r")\)" , r"\([A-Z]\)"]
        punctuations = ",-.:;"
        end_markers = "."
        current_index = 0
        regex_match_flag = 0
        max_context_length = 0

        contexts = []
        text = text.split()

        while (current_index < len(text)) :

            regex_match_flag = 0
            max_context_length = min(current_index + 512, len(text))
            context = text[current_index:max_context_length]
            if max_context_length == len(text) :
                contexts.append(context)
                current_index += len(context)
            else :
                i = len(context) - 1
                while i > (len(context) // 2) :
                    # if (re.match(level_wise_regex[0], context[i]) is not None) :
                    #     print("First Matched")
                    # if (re.match(level_wise_regex[1], context[i]) is not None) :
                    #     print("Second Matched")
                    # if (re.match(level_wise_regex[2], context[i]) is not None) :
                    #     print("Third Matched")
                    # if (re.match(level_wise_regex[3], context[i]) is not None) :
                    #     print("Fourth Matched")

                    if (re.match(level_wise_regex[0], context[i]) is not None) or \
                        (re.match(level_wise_regex[1], context[i]) is not None) or \
                        (re.match(level_wise_regex[2], context[i]) is not None) or \
                        (re.match(level_wise_regex[3], context[i]) is not None) :
                        if i != 0 and context[i-1][-1] in punctuations :
                            context = context[:i]
                            contexts.append(context)
                            current_index += i
                            regex_match_flag = 1
                            break
                    i -= 1

                if not regex_match_flag :
                    i = len(context) - 1
                    while (i > 0) :
                        if context[i][-1] in end_markers :
                            context = context[:i + 1]
                            contexts.append(context)
                            current_index += (i + 1)
                            break
                        i -= 1

            if len(contexts[-1]) <= 10 :
                del contexts[-1]

        return contexts

In [ ]:
import itertools

from nltk import sent_tokenize

import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

PIPELINE_SETTINGS = {
    #"model": "valhalla/t5-base-qg-hl",
    "model": "mrm8488/t5-base-finetuned-question-generation-ap",
    "ans_model": ["valhalla/t5-base-qa-qg-hl"]
}

class QGPipeline:

    def __init__(self, pipeline_settings: dict = PIPELINE_SETTINGS, use_cuda: bool = True) :

        self.model = AutoModelForSeq2SeqLM.from_pretrained(pipeline_settings['model'])
        self.tokenizer = AutoTokenizer.from_pretrained(pipeline_settings['model'], use_fast=False)

        self.ans_model = []
        self.ans_tokenizer = []
        for i in range(len(pipeline_settings['ans_model'])) :
            self.ans_model.append(AutoModelForSeq2SeqLM.from_pretrained(pipeline_settings['ans_model'][i]))
            self.ans_tokenizer.append(AutoTokenizer.from_pretrained(pipeline_settings['ans_model'][i], use_fast=False))

        self.device = "cuda" if torch.cuda.is_available() and use_cuda else "cpu"
        self.model.to(self.device)

        for i in range(len(self.ans_model)) :
            if self.ans_model[i] is not self.model:
                self.ans_model[i].to(self.device)

    def __call__(self, text : str):
        input_text = " ".join(text.split())
        answers = self._extract_answers(input_text)
        # print(sents)
        # print(answers)
        if len(answers) == 0:
          return []

        questions = self._generate_questions(answers, input_text)
        question_answers_list = []
        for question, answer in zip(questions, answers) :
            question_answers_list.append({'question': question, 'answer': answer})
        return question_answers_list

    def _extract_answers(self, context):
        inputs = self._prepare_inputs_for_ans_extraction(context)
        inputs = self._tokenize(inputs, padding=True, truncation=True)

        answers = []
        for i in range(len(self.ans_model)) :
            outs = self.ans_model[i].generate(
                input_ids=inputs['input_ids'].to(self.device), 
                attention_mask=inputs['attention_mask'].to(self.device), 
                max_length=32,
            )
            
            dec = [self.ans_tokenizer[i].decode(ids, skip_special_tokens=False) for ids in outs]
            decoded_output = [item.split('<sep>') for item in dec]
            decoded_output = [i[0] for i in decoded_output]
            answers.extend(decoded_output)
        
        for i in range(len(answers)) :
            answers[i] = answers[i].replace("<pad> ", "")

        answers = list(set(answers))
        return answers
 
    def _prepare_inputs_for_ans_extraction(self, text):
        sents = sent_tokenize(text)

        inputs = []
        for i in range(len(sents)):
            source_text = "extract answers:"
            for j, sent in enumerate(sents):
                if i == j:
                    sent = "<hl> %s <hl>" % sent
                source_text = "%s %s" % (source_text, sent)
                source_text = source_text.strip()
            
            source_text = source_text + " </s>"
            inputs.append(source_text)

        return inputs
  
    def _tokenize(self, inputs, padding=True, truncation=True, add_special_tokens=True, max_length=512):
        inputs = self.ans_tokenizer[0].batch_encode_plus(
            inputs, 
            max_length=max_length,
            add_special_tokens=add_special_tokens,
            truncation=truncation,
            padding="max_length" if padding else False,
            pad_to_max_length=padding,
            return_tensors="pt"
        )
        return inputs
    
    def _generate_questions(self, answers, context):
        questions = []
        for answer in answers :
            input_text = "answer: %s  context: %s </s>" % (answer, context)
            inputs = self._tokenize([input_text], padding=True, truncation=True)
        
            outs = self.model.generate(
                input_ids=inputs['input_ids'].to(self.device), 
                attention_mask=inputs['attention_mask'].to(self.device), 
                max_length=64,
                num_beams=4,
            )
            questions.extend([self.tokenizer.decode(ids, skip_special_tokens=True) for ids in outs])

        for i in range(len(questions)) :
            questions[i] = questions[i].replace("question: ", "")

        return questions


In [ ]:
#Importing ML Modules
import torch
#Importing the QA BERT Class and BERT Tokenizer
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

class QA() :
  def __init__(self) :
      #Defining the model with the tokenizer
      self.model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"
      self.model = AutoModelForQuestionAnswering.from_pretrained(self.model_name)
      self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)

  def answer(self, question, context) :
      #Tokenize the Question and the context
      #Concatenate the question and and context and add the special tokens
      #Encode the words into word embeddings
      input_ids = (self.tokenizer).encode(question, context)

      #Converting the token_ids back to tokens to print the answer at the end
      tokens = (self.tokenizer).convert_ids_to_tokens(input_ids)

      #Creating the segment_id list to specify the segment embedding to be added to the word embedding
      #A(0) segment - Question, B(1) segment - Answer Ref Text
      sep_index = input_ids.index((self.tokenizer).sep_token_id)	#separator token index
      num_seg_a = sep_index + 1                 			#Segment A
      num_seg_b = len(input_ids) - num_seg_a    			#Segment B
      segment_ids = [0]*num_seg_a + [1]*num_seg_b  

      #Running the model on the given question and answer ref text, specifying the segment_ids alongside
      #Fetching the start and end scores after taking dot product with the start and end vectors
      scores = self.model(torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]));
      start_scores = scores.start_logits;
      end_scores = scores.end_logits

      #Find the token index with the maximum start and end score by applying softmax activation (argmax function)
      start_index = torch.argmax(start_scores)
      end_index = torch.argmax(end_scores)

      #If the start index is more than the end index
      if(start_index > end_index) :
          return "No answer"

      #Processing the subword characters added by BERT to get a well organised answer
      answer = tokens[start_index]
      for i in range(start_index + 1, end_index + 1) :
          #subword token is added to the previous token to complete a word
          if tokens[i][0:2] == '##':
              answer += tokens[i][2:]

          #else add the token directly to the answer with a whitespace
          else: 
              answer += ' ' + tokens[i]

      #returning the answer
      # start_index = torch.max(start_index)
      # end_index = torch.max(end_index)
      start_index = start_index.item()
      end_index = end_index.item()
      return (answer, start_index, end_index)

In [ ]:
# %ls
import os
x = []
for i in os.listdir("./") :
  if i[-4:] == '.pdf' :
    x.append(i)
print(x)

[]


In [ ]:
# qg = QGPipeline()
qa = QA()

In [ ]:
import os
import sys 
sys.path += ['../']
import json

def get_questions_and_answers_given_the_context(context) :
    generated_questions = qg(context)    
    qas = []
    for i in range(len(generated_questions)) :
        question = generated_questions[i]['question']
        answer = qa.answer(question, context)
        # ans_startindex = context.find(answer)
        # ans_endindex = ans_startindex + len(answer)
        entry = {'question': question, 'answer': [{'ans_startindex': answer[1], 'ans_endindex': answer[2]}]}
        qas.append(entry)

    return qas



# legal_docs_names = os.listdir('../legal_docs/')
legal_docs_names = x
dataset = dict()

dataset['doccnt'] = len(legal_docs_names)
dataset['data'] = []
# for i in range(len(legal_docs_names)) :
for i in range(1) :
    dataset['data'].append({'title': legal_docs_names[i], 'paragraphs': []})

    # legal_docs_names[i] = os.path.abspath(os.path.join(os.path.dirname(os.path.dirname( \
    #     os.path.abspath(__file__))), "legal_docs", legal_docs_names[i]))

    contexts = TextProcessingAndContextCreation.get_context_chunks(legal_docs_names[i])
    contexts = contexts[-5:-1]
    for context in contexts :
        qas = get_questions_and_answers_given_the_context(context)
        dataset['data'][i]['paragraphs'].append({'context': context, 'qas': qas})


print(dataset)

with open("dataset.json", 'w') as outfile :
    json.dump(dataset, outfile, indent=4)

# print(dataset)

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:175: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."


{'doccnt': 2, 'data': [{'title': 'Technical Education Act, 1987.pdf', 'paragraphs': [{'context': 'Power to supersede the Council.- (1) If the Central Government is of the opinion that the Council is unable to perform, or has persistently made default in the performance of, the duty imposed on it by or under this Act or has exceeded or abused its powers, or has wilfully or without sufficient cause, failed to comply with any direction issued by the Central Government under section 20, the Central Government may, by notification in the Official Gazette, supersede the Council for such period as may be specified in the notification: Provided that before issuing a notification under this sub-section, the Central Government shall give a reasonable time to the Council to show cause why it should not be superseded and shall consider the explanation and objections, if any, of the Council. (2) Upon the publication of a notification under sub-section (1) superseding the Council,-  (a) all the memb